<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/4_KoBERT_emotion_finetune_ks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#변수 정의

In [1]:
ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_ks_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_ks.pt'

#실행 환경#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 1.3 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 6.6 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595726 sha256=b5532c94b87898050ccbc87580ed6e3c411880cde331b2a68eefa37c165bb804
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 7.0 MB/s 
     |████████████████████████████████| 769 kB 8.0 MB/s 
     |████████████████████████████████| 3.0 MB 49.1 MB/s 
     |████████████████████████████████| 895 kB 55.4 MB/s 


#KoBERT 다운로드#

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-g2sy1eij
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-g2sy1eij
     |████████████████████████████████| 4.9 MB 9.6 MB/s 
     |████████████████████████████████| 3.3 MB 57.5 MB/s 
     |████████████████████████████████| 596 kB 57.8 MB/s 
     |████████████████████████████████| 3.3 MB 48.2 MB/s 
     |████████████████████████████████| 61 kB 508 kB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=8ac75608991e7293c010ddab64e6e56fa4cef0b27b651437f6a5e7bdd474bfca
  Stored in directory: /tmp/pip-ephem-wheel-cache-3rmw19bo/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existi

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#데이터셋 전처리#

In [8]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [10]:
%pushd $TRAIN_DIR
tl = GetShellCmdStdOut('find | grep answer').split('\n')
train_json_list = [x.replace('./', '') for x in tl]
train_json_list.sort()
%popd

/content/drive/MyDrive/KAIST/SEP531/kor_ks_labeled/train
/content
popd -> /content


In [11]:
%pushd $VALID_DIR
vl = GetShellCmdStdOut('find | grep answer').split('\n')
valid_json_list = [x.replace('./', '') for x in vl]
valid_json_list.sort()
%popd

/content/drive/MyDrive/KAIST/SEP531/kor_ks_labeled/valid
/content
popd -> /content


In [12]:
train_json_list[0]

'DKCI20000009.json.answer'

In [13]:
!cat $TRAIN_DIR/{train_json_list[0]}

언니 언니야 강아지 키운다고 안 했나?	언니 언니야 강아지 키운다고 안 캤나?	1
아~ 나는 지금 강아지 키우는데 일곱 살이거든.	아~ 나는 지금 강아지 키우는데 일곱 살이거든.	2
음~ 요크셔테리어인데	음~ 요크셔테리어인데	4
내가 원래 요크셔테리어를 -초등학- 초등학교 때 키웠는데 교통사고로 죽었어.	내가 원래 요크셔테리어를 -초등학- 초등학교 때 키웠는데 교통사고로 죽었어.	3
그래 가지고 그때 그 애를 못 잊어서 요크셔테리어를 샀다가	그래 가지고 그때 그 애를 못 잊어서 요크셔테리어를 샀다가	4
키우게 됐어 그래서 지금 일곱 살인데 이~ 나한테 칠 개월에 입양 와 가지고 키우는데 뭐~	키우게 됐어 그래서 지금 일곱 살인데 이~ 나한테 칠 개월에 입양 와 가지고 키우는데 뭐~	4
잘 예뻐 요새는 다 포메라니안이랑 뭐~	잘 예뻐 요새는 다 포메라니안이랑 뭐~	5
나이가 꽤 있는데 -마- 많이 아프거나 그런 건 없니.	나이가 꽤 있는데 -마- 많이 아프거나 그런 건 없나.	3
걔가 지금 스케일링에 처음에 데려왔는데 이상하게 칠 개월인데 입 그~ 강아지도 유치가 있거든.	걔가 지금 스케일링에 처음에 데려왔는데 이상하게 칠 개월인데 입 그~ 강아지도 유치가 있거든.	0
이빨이 두 줄인 거라.	이빨이 두 줄인 거라.	6
빼줘야 되니.	빼줘야 되나.	3
어~ 근데 원래 빼줘야 되는데 그 이빨이	어~ 근데 원래 빼줘야 되는데 그 이빨이	6
유치랑 이빨 사이에 꺼멓게 그~	유치랑 이빨 사이에 꺼멓게 그~	6
이~ 이~ -석- 치석이 많이 꼈더라고 근데 그거를 나중에 내가 어쨌든	이~ 이~ -석- 치석이 많이 꼈더라고 근데 그거를 나중에 내가 어쨌든	1
나중에 중성화를 할 때 그 사람이 이~ 이빨을 유치를 같이 해 가지고 뽑아 줬어 근데 스케일링을 안 해준 거라.	나중에 중성화를 할 때 그 사람이 이~ 이빨을 유치를 같이 해 가지고 뽑아 줬어 근데 스케일링을 안 해준 거라.	0
그 상태로 계속 있다 보니까 그리고 나도 얘한테 양치질을 자주 못 시켜 주고 하다가 이빨이 너

In [14]:
import pandas as pd

chatbot_data = pd.read_csv('%s/%s' % (TRAIN_DIR, train_json_list[0]), sep='\t', header=None)

chatbot_data

,0,1,2
0,언니 언니야 강아지 키운다고 안 했나?,언니 언니야 강아지 키운다고 안 캤나?,1
1,아~ 나는 지금 강아지 키우는데 일곱 살이거든.,아~ 나는 지금 강아지 키우는데 일곱 살이거든.,2
2,음~ 요크셔테리어인데,음~ 요크셔테리어인데,4
3,내가 원래 요크셔테리어를 -초등학- 초등학교 때 키웠는데 교통사고로 죽었어.,내가 원래 요크셔테리어를 -초등학- 초등학교 때 키웠는데 교통사고로 죽었어.,3
4,그래 가지고 그때 그 애를 못 잊어서 요크셔테리어를 샀다가,그래 가지고 그때 그 애를 못 잊어서 요크셔테리어를 샀다가,4
...,...,...,...
415,조카만 보고 살까 이 생각도 하고 내가 막 뭐~ 어차피,조카만 보고 살까 이 생각도 하고 내가 막 뭐~ 어차피,0
416,물려줄 자식 없게 되면은 조카한테 막 해 줘야 되니까,물려줄 자식 없게 되면은 조카한테 막 해 줘야 되니까,6
417,늙더라도 늙어 병원에 한 번씩 들다 봐 주지 않을까 이런 생각도 하고,늙더라도 늙어 병원에 한 번씩 들다 봐 주지 않을까 이런 생각도 하고,4
418,쓸모 없는 생각 가끔 하지.,쓰잘데기 없는 생각 가끔 하지.,4


In [15]:
def GenDataset(DIR, json_list):
  retval = []

  for fn in json_list:
    #print('%s\n' % (fn))
    fd = pd.read_csv('%s/%s' % (DIR, fn), sep='\t', header=None)
    # [1]데이터가 사투리, [2]가 감정
    for q, label in zip(fd[1], fd[2])  :
      #print('%s, %s\n' % (q, label))
      data = []
      data.append(q)
      data.append(str(label))
      retval.append(data)

  return retval

In [20]:
dataset_train = GenDataset(TRAIN_DIR, train_json_list)

In [21]:
dataset_valid = GenDataset(VALID_DIR, valid_json_list)

In [23]:
dataset_train = dataset_train[0:45000]
dataset_valid = dataset_valid[0:15000]

In [24]:
print(len(dataset_train))
print(len(dataset_valid))

45000
15000


#KoBERT 입력 데이터로 변환#

In [32]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [33]:
# Setting parameters
max_len = 64
batch_size = 64 # origin : 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [34]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_valid = BERTDataset(dataset_valid, 0, 1, tok, max_len, True, False)

using cached model


In [35]:
# 첫번째는 패딩된 시퀀스, 두번째는 길이와 티입, 세번째는 어텐션 마스크 시퀀스
data_train[0]

(array([   2, 3241, 5770, 3241, 5770, 6844,  807, 6797, 7318, 4693, 7010,
        5782, 5439, 3135,  517,    0, 5655,  633,    3,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(19, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [36]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_valid, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#KoBERT 학습모델 만들기#

In [37]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [38]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)
model.load_state_dict(torch.load('/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_added.pt'), strict=False)
model.eval()

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

#KoBERT 모델 학습시키기#

In [39]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/704 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.9053072333335876 train acc 0.6875
epoch 1 batch id 201 loss 0.7572425603866577 train acc 0.7426150497512438
epoch 1 batch id 401 loss 1.0817888975143433 train acc 0.7232309850374065
epoch 1 batch id 601 loss 1.165995717048645 train acc 0.7062707986688852
epoch 1 train acc 0.7002840909090909


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.7660017730496453


  0%|          | 0/704 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.1677221655845642 train acc 0.953125
epoch 5 batch id 201 loss 0.08651387691497803 train acc 0.9640858208955224
epoch 5 batch id 401 loss 0.19923977553844452 train acc 0.96723036159601
epoch 5 batch id 601 loss 0.07735122740268707 train acc 0.968698003327787
epoch 5 train acc 0.9688609730113636


  0%|          | 0/235 [00:00<?, ?it/s]

epoch 5 test acc 0.7763741134751773


In [40]:
torch.save(model.state_dict(), SAVEPOINT_PATH)